<a href="https://colab.research.google.com/github/SamuelPassamani/XCam/blob/main/xcam-rec/XCam_REC_Launcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 XCam REC Engine - Lançador Modular
---
**@titulo:** XCam_REC_Launcher.ipynb

**@author:** Samuel Passamani / Um Projeto do Estudio A.Sério [AllS Company]

**@info:** https://aserio.work/

**@version:** 1.5.0

**@lastupdate:** 2025-08-27

**@description:** Este notebook serve como uma interface de usuário amigável para o projeto XCam Rec. Ele automatiza o processo de clonar (ou atualizar) o repositório, instalar as dependências necessárias e executar o script principal (`main.py`) com parâmetros configuráveis através de um formulário interativo. Combina a simplicidade de uso do Colab com a robustez de uma arquitetura de software modular.

### ⚙️ Etapa 1: Preparação do Ambiente

Esta célula executa três ações essenciais:
1.  **Monta o seu Google Drive**: Para que o nosso script possa aceder à pasta de configuração e salvar os logs e arquivos.
2.  **Clona ou Atualiza o Repositório**: Se o projeto `XCam` ainda não existir no seu Drive, ele será clonado do GitHub. Se já existir, ele será atualizado para a versão mais recente.
3.  **Instala as Dependências**: Lê o arquivo `requirements.txt` e instala todas as bibliotecas Python necessárias para a execução.

In [ ]:
from google.colab import drive
import os
import sys

# Monta o Google Drive
print("🛰️  Montando o Google Drive...")
drive.mount('/content/drive')
print("✅ Google Drive montado em /content/drive.")

# Define o caminho base e o caminho do projeto
GDRIVE_BASE_PATH = "/content/drive/MyDrive/Projetos/XCam/XCam Drive/XCam.Drive/"
PROJECT_PATH = os.path.join(GDRIVE_BASE_PATH, "XCam")
REC_ENGINE_PATH = os.path.join(PROJECT_PATH, "xcam-rec")

# Adiciona o diretório do motor de gravação ao path do sistema para importar o config
if REC_ENGINE_PATH not in sys.path:
    sys.path.append(REC_ENGINE_PATH)

# Verifica se o projeto já existe para clonar ou atualizar
if not os.path.exists(PROJECT_PATH):
    print(f"📂 Projeto não encontrado. Clonando de https://github.com/SamuelPassamani/XCam.git para {GDRIVE_BASE_PATH}...")
    os.chdir(GDRIVE_BASE_PATH)
    !git clone https://github.com/SamuelPassamani/XCam.git
else:
    print("🔄 Projeto já existe. Atualizando para a versão mais recente...")
    os.chdir(PROJECT_PATH)
    !git pull origin main

# Navega para o diretório do nosso motor de gravação
print(f"🧭 Navegando para o diretório do motor de gravação: {REC_ENGINE_PATH}")
os.chdir(REC_ENGINE_PATH)

# Instala as dependências do projeto
print("🐍 Instalando dependências a partir de requirements.txt...")
!pip install -q -r requirements.txt
print("✅ Ambiente pronto para execução.")

### ▶️ Etapa 2: Executar o XCam Rec Engine

Utilize o formulário abaixo para configurar os parâmetros da execução. Se deixar um campo em branco, os valores padrão do ficheiro `config.py` serão utilizados. Após ajustar os valores, execute esta célula para iniciar o processo de gravação.

In [ ]:
#@title Iniciar o Motor de Gravação
#@markdown ---
#@markdown ### ⚙️ Configurações de Execução:
#@markdown Deixe os campos em branco para usar os valores padrão definidos em `config.py`.

import config
import shutil
import glob
import os

# --- Limpeza da pasta de gravações temporárias antes da execução ---
for file in glob.glob(os.path.join(config.TEMP_RECORDS_PATH, "*")):
    try:
        os.remove(file)
    except Exception as e:
        print(f"Erro ao remover {file}: {e}")

# --- Definição dos Campos do Formulário ---
Pagina = 1 #@param {type:"raw"}
Limite = 1000 #@param {type:"raw"}
Trabalhadores = 50 #@param {type:"raw"}
Duracao_Maxima_s = 7200 #@param {type:"raw"}
Duracao_Minima_s = 420 #@param {type:"raw"}
Pais = "" #@param {type:"string"}
#@markdown Caminho da Marca d'Água (opcional)
Watermark_Path = "" #@param {type:"string"}
#@markdown Largura Máxima da Marca d'Água (px, opcional)
Watermark_Width = "" #@param {type:"string"}

# --- Validação e Definição de Valores Padrão ---
print("⚙️  Validando configurações...")

final_page = int(Pagina) if Pagina is not None and str(Pagina).strip() != '' else config.DEFAULT_EXECUTION_SETTINGS['API_PARAMS']['page']
final_limit = int(Limite) if Limite is not None and str(Limite).strip() != '' else config.DEFAULT_EXECUTION_SETTINGS['API_PARAMS']['limit']
final_workers = int(Trabalhadores) if Trabalhadores is not None and str(Trabalhadores).strip() != '' else 5
final_max_duration = int(Duracao_Maxima_s) if Duracao_Maxima_s is not None and str(Duracao_Maxima_s).strip() != '' else config.DEFAULT_EXECUTION_SETTINGS['MAX_DURATION_SECONDS']
final_min_duration = int(Duracao_Minima_s) if Duracao_Minima_s is not None and str(Duracao_Minima_s).strip() != '' else config.DEFAULT_EXECUTION_SETTINGS['MIN_DURATION_SECONDS']
final_country = Pais.strip() if Pais else config.DEFAULT_EXECUTION_SETTINGS['API_PARAMS']['country']
final_watermark_path = Watermark_Path.strip() if Watermark_Path.strip() else getattr(config, 'WATERMARK_IMAGE_PATH', '')
final_watermark_width = int(Watermark_Width) if Watermark_Width.strip() else getattr(config, 'WATERMARK_MAX_WIDTH', 180)

print("    - Página da API: ", final_page)
print("    - Limite de Modelos: ", final_limit)
print("    - Trabalhadores: ", final_workers)
print(f"    - Duração Máxima: {final_max_duration}s")
print(f"    - Duração Mínima: {final_min_duration}s")
print(f"    - País: '{final_country if final_country else 'Todos'}'")
print(f"    - Caminho da Marca d'Água: {final_watermark_path}")
print(f"    - Largura Máxima da Marca d'Água: {final_watermark_width}px")

command = (
    f"python main.py "
    f"--page {final_page} "
    f"--limit {final_limit} "
    f"--workers {final_workers} "
    f"--max-duration {final_max_duration} "
    f"--min-duration {final_min_duration}"
)

if final_country:
    command += f" --country {final_country}"

command += f" --watermark-path '{final_watermark_path}' --watermark-width {final_watermark_width}"

print("-"*50)
print(f"🚀 Executando comando: {command}")
print("-"*50)

!{command}

---
### 📜 Histórico de Mudanças e Roadmap

**@log de mudanças:**
* **2025-08-27 (v1.5.0):**
    * MELHORIA: Agora a pasta de gravações temporárias é limpa antes de cada execução (garantindo que apenas vídeos da execução corrente ficarão na pasta).
* **2025-08-27 (v1.4.0):**
    * FEATURE: Adicionados campos para caminho e largura da marca d'água ao formulário do notebook.
    * O comando de execução agora passa os argumentos `--watermark-path` e `--watermark-width`.
* **2025-07-14 (v1.3.0):**
    * FEATURE: Adicionados os campos `Duração Máxima (s)` e `Duração Mínima (s)` ao formulário.
    * REFACTOR: O formulário agora busca os valores padrão diretamente do `config.py` se os campos forem deixados em branco.
    * REFACTOR: Os rótulos do formulário foram traduzidos para Português e capitalizados.
    * REFACTOR: O comando de execução foi atualizado para usar os argumentos `--max-duration` e `--min-duration`.
* **2025-07-13 (v1.2.0):**
    * MELHORIA: Adicionada lógica para usar valores padrão se os campos do formulário fossem deixados em branco.
* **2025-07-13 (v1.1.0):**
    * CORREÇÃO: O argumento de linha de comando `--pages` foi corrigido para `--page`.
* **2025-07-13 (v1.0.0):**
    * Criação inicial do `XCam_Rec_Launcher.ipynb`.

**@roadmap futuro:**
* Adicionar um campo de formulário para permitir a gravação de uma lista de usuários específicos.
* Integrar uma célula opcional para exibir o conteúdo do arquivo de log (`xcam_recorder.log`) diretamente no notebook.
* Criar uma célula para fazer o "commit" e "push" das alterações nos arquivos `rec.json`.